<div style="width: 30%; float: right; margin: 10px; margin-right: 5%;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/FHNW_Logo.svg/2560px-FHNW_Logo.svg.png" width="500" style="float: left; filter: invert(50%);"/>
</div>

<h1 style="text-align: left; margin-top: 10px; float: left; width: 60%;">
    SAN Projekt:<br> Schweizer Offshore Firmen
</h1>

<p style="clear: both; text-align: left;">
    Bearbeitet durch Florin Barbisch, Gabriel Torres Gamez und Tobias Buess im FS 2024.
</p>

Wir führen eine Voranalyse für das Bundesamt für Statistik durch, um die kürzlich aufgetretenen Leaks aus den Offshore Papers zu untersuchen. 

Diese Analyse zielt darauf ab, Umfang und Natur der Verbindungen in Schweizer Offshore-Strukturen zu ermitteln. Wir verwenden dafür Daten aus der [Offshore Leaks Database](https://offshoreleaks.icij.org/), um mögliche Muster, wichtige Personen aufzudecken, die für die Steuerbehörden oder Regulierungsorgane von Interesse sein könnten. 

Unsere Arbeit umfasst eine detaillierte Prüfung der betroffenen Entitäten. Dies wird es dem Bundesamt für Statistik ermöglichen, fundierte Entscheidungen zur weiteren Untersuchung und möglichen Massnahmen zu treffen.

## Imports und Einstellungen

In [1]:
import os
import re
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt

# if cugraph is installed, import it
cugraph_installed = False
try:
    import cugraph as cnx
    cugraph_installed = True
except:
    pass

print("Python Environment:")
print(f" | Python version: {os.sys.version}")
print(f" | Numpy version: {np.__version__}")
print(f" | Pandas version: {pd.__version__}")
print(f" | NetworkX version: {nx.__version__}")
print(f" | Matplotlib version: {matplotlib.__version__}")
print(
    f" | CuGraph version: {cnx.__version__}"
    if cugraph_installed
    else " | CuGraph not installed"
)
print()
print("Ressources:")
print(f" | CPU: {os.cpu_count()} cores")

PAPERS = "Pandora Papers"
GRAPH_PATH = f"./data/{PAPERS.lower().replace(' ', '_')}_graph.gexf"

Python Environment:
 | Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
 | Numpy version: 1.26.4
 | Pandas version: 2.2.1
 | NetworkX version: 3.3
 | Matplotlib version: 3.8.4
 | CuGraph version: 24.04.00

Ressources:
 | CPU: 24 cores


## Inhalt
1. Wie sehen die Daten aus? Was für Informationen können wir daraus ziehen?
2. Grobe Metriken zu Schweizer Officers und deren Offshore Firmen.
3. ...

# Daten laden

In [2]:
def remove_special_characters(text):
    # Stellen Sie sicher, dass der Text ein String ist
    if isinstance(text, str):
        # Entfernt alles außer Buchstaben, Ziffern, Leerzeichen und grundlegenden Satzzeichen
        return re.sub(r"[^\w\s,.]", "", text)
    return text

In [3]:
if not os.path.exists(GRAPH_PATH):
    # read all the data
    addressNodes = (
        pd.read_csv("./data/nodes-addresses.csv", low_memory=False, index_col=0)
        .astype(str)
        .map(remove_special_characters)
    )
    addressNodes["node_type"] = "Address"

    entityNodes = (
        pd.read_csv("./data/nodes-entities.csv", low_memory=False, index_col=0)
        .astype(str)
        .map(remove_special_characters)
    )
    entityNodes["node_type"] = "Entity"

    intermediaryNodes = (
        pd.read_csv("./data/nodes-intermediaries.csv", low_memory=False, index_col=0)
        .astype(str)
        .map(remove_special_characters)
    )
    intermediaryNodes["node_type"] = "Intermediary"

    officerNodes = (
        pd.read_csv("./data/nodes-officers.csv", low_memory=False, index_col=0)
        .astype(str)
        .map(remove_special_characters)
    )
    officerNodes["node_type"] = "Officer"

    nodes_others = (
        pd.read_csv("./data/nodes-others.csv", low_memory=False, index_col=0)
        .astype(str)
        .map(remove_special_characters)
    )
    nodes_others["node_type"] = "Other"

    relationships = (
        pd.read_csv("./data/relationships.csv", low_memory=False)
        .set_index(["node_id_start", "node_id_end"])
        .astype(str)
        .map(remove_special_characters)
    )

    # filter all nodes and relationships that are not from the Pandora Papers
    addressNodes = addressNodes[addressNodes["sourceID"].str.contains(PAPERS)]
    entityNodes = entityNodes[entityNodes["sourceID"].str.contains(PAPERS)]
    intermediaryNodes = intermediaryNodes[
        intermediaryNodes["sourceID"].str.contains(PAPERS)
    ]
    officerNodes = officerNodes[officerNodes["sourceID"].str.contains(PAPERS)]
    nodes_others = nodes_others[nodes_others["sourceID"].str.contains(PAPERS)]

    # alternatively, get all nodeIDs from all filtered nodes and remove relationships with nodeIDs that are not in this list
    allNodeIDs = pd.concat(
        [addressNodes, entityNodes, intermediaryNodes, officerNodes, nodes_others]
    ).index
    relationships = relationships[
        relationships.index.get_level_values(0).isin(allNodeIDs)
        & relationships.index.get_level_values(1).isin(allNodeIDs)
    ]

    # create the graph
    G = nx.MultiDiGraph()
    G.add_nodes_from(
        [(key, value) for key, value in addressNodes.to_dict("index").items()]
    )
    G.add_nodes_from(
        [(key, value) for key, value in entityNodes.to_dict("index").items()]
    )
    G.add_nodes_from(
        [(key, value) for key, value in intermediaryNodes.to_dict("index").items()]
    )
    G.add_nodes_from(
        [(key, value) for key, value in officerNodes.to_dict("index").items()]
    )
    G.add_nodes_from(
        [(key, value) for key, value in nodes_others.to_dict("index").items()]
    )
    G.add_edges_from(
        [
            (*relationships.index[i], value)
            for i, value in enumerate(relationships.to_dict(orient="records"))
        ]
    )

    # remove all the dataframes
    del addressNodes
    del entityNodes
    del intermediaryNodes
    del officerNodes
    del nodes_others
    del relationships

    # save the graph
    nx.write_gexf(G, GRAPH_PATH)

    # remove the graph
    del G

G = nx.read_gexf(GRAPH_PATH)

## Wie sehen die Daten aus? Was für Informationen können wir daraus ziehen?

## Grobe Metriken zu schweizer Officers und deren Offshore Firmen.

## ...